In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 27.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 4.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3

In [4]:
import os
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA


In [24]:
os.environ["GROQ_API_KEY"] = "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"

In [25]:
# fetch the pdf from url
import requests
url = "https://cs.nyu.edu/courses/summer16/CSCI-UA.0002-002/slides/Python_Functions.pdf"

response = requests.get(url)
response


<Response [200]>

In [26]:
# save the pdf local file
with open("Python_inbuildfunctions.pdf", "wb") as f:
    f.write(response.content)


In [27]:
#load the document
loader = UnstructuredPDFLoader("Python_inbuildfunctions.pdf")

In [28]:
documents = loader.load()
documents

[Document(metadata={'source': 'Python_inbuildfunctions.pdf'}, page_content='+\n\nWorking with Functions in Python\n\nIntroduction to Programming - Python\n\n+\n\nFunctions\n\n+\n\nFunctions\n\nn\u202f A function is a group of statements that exist within a\n\nprogram for the purpose of performing a specific task\n\nn\u202f Since the beginning of the semester we have been using a\n\nnumber of Python’s built-in functions, including: n\u202f print()\n\nn\u202f range()\n\nn\u202f len()\n\nn\u202f random.randint()\n\nn\u202f … etc\n\n+\n\nFunctions\n\nn\u202f Most programs perform tasks that are large enough to be\n\nbroken down into subtasks\n\nn\u202f Because of this, programmers often organize their programs into smaller, more manageable chunks by writing their own functions\n\nn\u202f Instead of writing one large set of statements we can break down a program into several small functions, allowing us to “divide and conquer” a programming problem\n\n+\n\nDefining Functions\n\nn\u202f Func

In [29]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

In [30]:
text_splitter

In [31]:
texts = text_splitter.split_documents(documents)

In [32]:
type(texts)

list

In [33]:
texts[2]

Document(metadata={'source': 'Python_inbuildfunctions.pdf'}, page_content='+\n\nFlow of Execution\n\n+\n\nFlow of Execution\n\n+\n\nFlow of Execution\n\n+\n\nFlow of Execution\n\n+\n\nFlow of Execution\n\n+\n\nFlow of Execution\n\n+\n\nFlow of Execution\n\n+\n\nFlow of Execution\n\n+\n\nFlow of Execution\n\n+\n\nFlow of Execution\n\n+\n\nFlow of Execution – With Functions\n\n+\n\nFlow of Execution with Functions\n\n+\n\nFlow of Execution with Functions\n\n+\n\nFlow of Execution with Functions\n\n+\n\nFlow of Execution with Functions\n\n+\n\nFlow of Execution with Functions\n\n+\n\nFlow of Execution with Functions\n\n+\n\nFlow of Execution with Functions\n\n+\n\nFlow of Execution with Functions\n\n+\n\nFlow of Execution with Functions\n\n+\n\nFlow of Execution with Functions\n\n+\n\nMultiple Functions\n\n+\n\nMultiple functions\n\ndef hello():\n\nprint (“Hello there!”)\n\ndef goodbye():\n\nprint (“See ya!”)\n\nhello()\n\ngoodbye()\n\n+\n\nCalling functions inside functions\n\ndef main()

In [34]:


from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

In [35]:
persist_directory = "vector_db"

In [16]:
vectordb = Chroma.from_documents(texts, embeddings, persist_directory=persist_directory)

In [17]:
# retrival

retriever = vectordb.as_retriever()

In [36]:
llm = ChatGroq(model="Llama3-8b-8192", temperature=0)

In [37]:
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)

In [38]:
query = 'what are thr inbuild function from this pdf'
response = qa_chain.invoke({"query":query})

In [39]:
print(response)

{'query': 'what are thr inbuild function from this pdf', 'result': 'Based on the provided PDF, the following are the built-in functions mentioned:\n\n1. `print()`\n2. `input()`\n3. `range()`\n4. `random.randint()`\n5. `time()`', 'source_documents': [Document(metadata={'source': 'Python_inbuildfunctions.pdf'}, page_content='they hit a bullseye. If the distance is >= 10, they missed!\n\n+\n\nIPO Notation\n\nn\u202f As you start writing more advanced functions you should\n\nthink about documenting them based on their Input, Processing and Output (IPO)\n\nn\u202f Example:\n\n# function: add_ages\u2028 # input: age1 (integer), age2 (integer)\u2028 # processing: combines the two integers\u2028 # output: returns the combined value\u2028 def add_ages(age1, age2):\u2028 sum = age1+age2\u2028 return sum\n\n+\n\nReturning Boolean Values\n\nn\u202f Boolean values can drastically\n\nsimplify decision and repetition structures\n\nn\u202f Example:\n\nn\u202f Write a program that asks the user\n\nfor 

In [41]:
query1 = "what is the pyhton function"
response1 = qa_chain.invoke({"query":query1})
print(response1)

{'query': 'what is the pyhton function', 'result': 'In Python, a function is a block of code that can be executed multiple times from different parts of your program. It\'s a way to group a set of statements together to perform a specific task. Functions can take arguments, which are values passed to the function when it\'s called, and they can return values, which are sent back to the caller.\n\nFunctions are useful for several reasons:\n\n1. Reusability: You can use a function multiple times in your program without having to rewrite the code.\n2. Modularity: Functions help organize your code into smaller, more manageable chunks.\n3. Code reuse: You can reuse functions in different parts of your program or even in other programs.\n4. Easier debugging: With functions, you can isolate and debug specific parts of your code more easily.\n\nIn Python, you can define a function using the `def` keyword, followed by the function name and arguments in parentheses. For example:\n```\ndef greet(

In [42]:
print(response["result"])

Based on the provided PDF, the following are the built-in functions mentioned:

1. `print()`
2. `input()`
3. `range()`
4. `random.randint()`
5. `time()`


In [43]:
query = "give me summary of all function from this pdf?"
response = qa_chain.invoke({"query":query})
print(response["result"])
print("*"*40)

Based on the provided PDF, here is a summary of the functions discussed:

1. Defining Functions:
	* Functions are groups of statements that perform a specific task.
	* They can be used to break down a program into smaller, more manageable chunks.
	* Functions must be named and created before they can be used.
2. IPO Notation:
	* Input: The data that is passed to the function.
	* Processing: The actions performed on the input data.
	* Output: The result of the function.
3. Returning Values:
	* Functions can return a single value using the `return` statement.
	* Functions can also return multiple values using the `return` statement with multiple values.
4. Returning Boolean Values:
	* Boolean values can be used to simplify decision and repetition structures.
	* Example: Writing a program that asks the user for a part number and only accepts part numbers that are on a specific list.
5. Passing Arguments to a Function:
	* When passing arguments, the function definition should specify the v